In [2]:
# !pip install PyPDF2
# !pip install nltk
# !pip install wordcloud
# !pip install PyMuPDF
# from spacy.cli import download
# download("en_core_web_sm")
# !pip install pycryptodome
# !pip install bertopic
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
# !pip install keybert
# import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')
# !pip install -U sentence-transformers

In [54]:
import PyPDF2
import string
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import spacy
import re
import matplotlib.pyplot as plt
from collections import Counter
import fitz  # PyMuPDF
from wordcloud import WordCloud
from spacy import displacy
from nltk.tokenize import word_tokenize
from bertopic import BERTopic
import os
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
stop = set(stopwords.words("english"))
from nltk import sent_tokenize, word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.util import ngrams
nlp = spacy.load("en_core_web_sm")
import ast
from sentence_transformers import SentenceTransformer, util
Sbert = SentenceTransformer('all-MiniLM-L6-v2')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading and Preprocessing the Corpus

## Extracting PMBOK

In [ ]:
pdf_file_path = "/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/PMBOK6-2017.pdf"
pdf_document = fitz.open(pdf_file_path)
extracted_text = ''
for page_num in range(430,493): #395+35 -> 458+35
    page = pdf_document[page_num]
    page_text = page.get_text()
    extracted_text += page_text
pdf_document.close()
# extracted_text

## Cleaning PMBOK

In [ ]:
def clean_pmbok(extracted_text):
    patterns_and_replacements = [
        (r'Not For Distribution, Sale or Reproduction\.', ''),
        (r'Part 1 - Guide', ''),
        (r'\d{3}', ''),
        (r'(?<![a-zA-Z])u(?!.)', ''),
        (r'u ', ''),
        (r'\bn\b',''),
        (r'ﬁ', 'fi'),
        (r'\(', ''),
        (r'\)', ''),
        (r',', ''),
        (r'`', '')
        #(r'\n','')
    ]
    for pattern, replacement in patterns_and_replacements:
        extracted_text = re.sub(pattern, replacement, extracted_text)

    return extracted_text

## Section Extraction and Analysis

In [ ]:
def extract_table_of_contents(pdf_path, start_page, end_page):
    pdf_document = fitz.open(pdf_path)
    table_of_contents = ""
    for page_num in range(start_page - 1, end_page):
        page = pdf_document.load_page(page_num)
        table_of_contents += page.get_text()

    pdf_document.close()
    return table_of_contents

table_of_contents_text = extract_table_of_contents(pdf_file_path, 11, 12)
print(table_of_contents_text[50:1000])


4 Develop Team................................................................................................................336
9.4.1 Develop Team: Inputs .......................................................................................339
9.4.2 Develop Team: Tools and Techniques ..............................................................340
9.4.3 Develop Team: Outputs ....................................................................................343
9.5 Manage Team................................................................................................................345
9.5.1 Manage Team: Inputs .......................................................................................347
9.5.2 Manage Team: Tools and Techniques ..............................................................348
9.5.3 Manage Team: Outputs ....................................................................................350
9.6 Contro


### Extracting RISK MANAGEMENT only

In [ ]:
lines = table_of_contents_text.split('\n')
start = lines.index('11. PROJECT RISK MANAGEMENT .............................................................................................395')
finish = lines.index('12. PROJECT PROCUREMENT MANAGEMENT ............................................................................459')
lines = lines[start:finish]
print(lines)

['11. PROJECT RISK MANAGEMENT .............................................................................................395', '11.1 Plan Risk Management ..............................................................................................401', '11.1.1 Plan Risk Management: Inputs ......................................................................402', '11.1.2 Plan Risk Management: Tools and Techniques .............................................404', '11.1.3 Plan Risk Management: Outputs ...................................................................405', 'Not For Distribution, Sale or Reproduction.', 'VIII ', 'Table of Contents', '11.2 Identify Risks ..............................................................................................................409', '11.2.1 Identify Risks: Inputs .....................................................................................411', '11.2.2 Identify Risks: Tools and Techniques .....................................

### Cleaning Table of Contents

In [ ]:
table_of_contents_text_clean = ''.join(lines)
table_of_contents_text_clean = table_of_contents_text_clean.replace('Not For Distribution, Sale or Reproduction','')
table_of_contents_text_clean = table_of_contents_text_clean.replace('Table of Contents','')

### Get titles

In [ ]:
title_pattern = r'\s+([A-Za-z\s:]+)'
titles = []
title_matches = re.findall(title_pattern, table_of_contents_text_clean)
titles.extend(title_matches)
print(titles)

['PROJECT RISK MANAGEMENT ', 'Plan Risk Management ', 'Plan Risk Management: Inputs ', 'Plan Risk Management: Tools and Techniques ', 'Plan Risk Management: Outputs ', 'Identify Risks ', 'Identify Risks: Inputs ', 'Identify Risks: Tools and Techniques ', 'Identify Risks: Outputs ', 'Perform Qualitative Risk Analysis ', 'Perform Qualitative Risk Analysis: Inputs ', 'Perform Qualitative Risk Analysis: Tools and Techniques ', 'Perform Qualitative Risk Analysis: Outputs ', 'Perform Quantitative Risk Analysis ', 'Perform Quantitative Risk Analysis: Inputs ', 'Perform Quantitative Risk Analysis: Tools and Techniques ', 'Perform Quantitative Risk Analysis: Outputs ', 'Plan Risk Responses ', 'Plan Risk Responses: Inputs ', 'Plan Risk Responses: Tools and Techniques ', 'Plan Risk Responses: Outputs ', 'Implement Risk Responses ', 'Implement Risk Responses: Inputs ', 'Implement Risk Responses: Tools and Techniques ', 'Implement Risk Responses: Outputs', 'Monitor Risks ', 'Monitor Risks: Inputs '

### Get Page Numbers

In [ ]:
page_number_pattern = r'\d{3}'
page_numbers = []
page_num_matches = re.findall(page_number_pattern, table_of_contents_text_clean)
page_numbers.extend(page_num_matches)
print(page_numbers)

['395', '401', '402', '404', '405', '409', '411', '414', '417', '419', '421', '422', '427', '428', '430', '431', '436', '437', '439', '441', '447', '449', '450', '451', '451', '453', '455', '456', '457']


### Get Sections

In [ ]:
section_pattern = r'^(\d+(\.\d{1,3})*)'
sections = []
for line in lines:
    section_match = re.search(section_pattern, line)
    if section_match:
        sections.append(section_match.group(1))
print(sections)

['11', '11.1', '11.1.1', '11.1.2', '11.1.3', '11.2', '11.2.1', '11.2.2', '11.2.3', '11.3', '11.3.1', '11.3.2', '11.3.3', '11.4', '11.4.1', '11.4.2', '11.4.3', '11.5', '11.5.1', '11.5.2', '11.5.3', '11.6', '11.6.1', '11.6.2', '11.6.3', '11.7', '11.7.1', '11.7.2', '11.7.3']


### Verifying Lengths for each list

In [ ]:
print(f' Sections : {len(sections)}')
print(f' Titles : {len(titles)}')
print(f' Page Numbers : {len(page_numbers)}')


 Sections : 29
 Titles : 29
 Page Numbers : 29


### Creating a Dataframe

In [ ]:
data = {"Title": titles, "Starting_Page_Number": page_numbers, "Section": sections}
df = pd.DataFrame(data)
df.sample(12)

,Title,Starting_Page_Number,Section
14,Perform Quantitative Risk Analysis: Inputs,430,11.4.1
6,Identify Risks: Inputs,411,11.2.1
11,Perform Qualitative Risk Analysis: Tools and T...,422,11.3.2
23,Implement Risk Responses: Tools and Techniques,451,11.6.2
17,Plan Risk Responses,437,11.5
25,Monitor Risks,453,11.7
9,Perform Qualitative Risk Analysis,419,11.3
15,Perform Quantitative Risk Analysis: Tools and ...,431,11.4.2
16,Perform Quantitative Risk Analysis: Outputs,436,11.4.3
26,Monitor Risks: Inputs,455,11.7.1


In [ ]:
df.head()

,Title,Starting_Page_Number,Section
0,PROJECT RISK MANAGEMENT,395,11
1,Plan Risk Management,401,11.1
2,Plan Risk Management: Inputs,402,11.1.1
3,Plan Risk Management: Tools and Techniques,404,11.1.2
4,Plan Risk Management: Outputs,405,11.1.3


### Adding 'Ending Page Number' Column

In [ ]:
page_numbers = df['Starting_Page_Number'].tolist()
end_pages = page_numbers
end_pages.pop(0)
end_pages.append('459')
print(end_pages)
df['Ending_Page_Number'] = end_pages

['401', '402', '404', '405', '409', '411', '414', '417', '419', '421', '422', '427', '428', '430', '431', '436', '437', '439', '441', '447', '449', '450', '451', '451', '453', '455', '456', '457', '459']


In [ ]:
df['Title'] = df['Title'].str.rstrip()

In [ ]:
df['Starting_Page_Number'] = df['Starting_Page_Number'].astype('int')
df['Ending_Page_Number'] = df['Ending_Page_Number'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Title                 29 non-null     object
 1   Starting_Page_Number  29 non-null     int64 
 2   Section               29 non-null     object
 3   Ending_Page_Number    29 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.0+ KB


In [ ]:
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number
0,PROJECT RISK MANAGEMENT,395,11,401
1,Plan Risk Management,401,11.1,402
2,Plan Risk Management: Inputs,402,11.1.1,404
3,Plan Risk Management: Tools and Techniques,404,11.1.2,405
4,Plan Risk Management: Outputs,405,11.1.3,409


### Content Extraction and Cleaning For Each Section

In [ ]:
def content_extraction_each_section(pdf_file,Starting_Page_Number,Ending_Page_Number):
    pdf_document = fitz.open(pdf_file)
    extracted_text = ""
    for page_num in range(Starting_Page_Number +35, Ending_Page_Number+35):
        page = pdf_document.load_page(page_num)
        extracted_text += page.get_text()
    clean_text = clean_pmbok(extracted_text)
    pdf_document.close()
    return clean_text

In [ ]:
def content_extraction_for_df(df):
    df['Content'] = df.apply(lambda x: content_extraction_each_section(pdf_file_path,x['Starting_Page_Number'],x['Ending_Page_Number']),axis=1)
    return df

In [ ]:
content_extraction_for_df(df)
df['Content'] = df['Content'].str.replace('\n','')
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
2,Plan Risk Management: Inputs,402,11.1.1,404,• Project charter11.1 Plan RiskManagementEn...
3,Plan Risk Management: Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNI...
4,Plan Risk Management: Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


In [ ]:
df.loc[2].Content

' • Project   charter11.1 Plan RiskManagementEnterprise/Organization4.1Develop ProjectCharter• Project charterProject documents• Stakeholder register• Enterprise environmental factors• Organizational process assetsProject ManagementPlanProject management plan• All componentsProjectManagementPlanProjectDocuments• Risk management planFigure 11-3. Plan Risk Management: Data Flow DiagramThe Plan Risk Management process should begin when a project is conceived and should be completed early in the project. It may be necessary to revisit this process later in the project life cycle for example at a major phase change or if the project scope changes significantly or if a subsequent review of risk management effectiveness determines that the Project Risk Management process requires modification.11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PROJECT CHARTERDescribed in Section 4.1.3.1. The project charter documents the high-level project description and boundaries high-level requirements and risks.

### Cleaning Extracted Content
Some Sections end at the same page as another Section begins so we're removing extra contents that doesnt belong to a specific Section [getting the content of each section only when we find the section and the title in the content]

*example : Section '11.1' Title : 'Plan Risk Management' Content : ' • Project   charter11.1 Plan Risk Management'* -> Cleaned Content : '11.1 Plan Risk Management'

In [ ]:
def extract_substring(row):
    start_string = row['Section'] + ' ' + row['Title'].upper()
    start_index = row['Content'].find(start_string)
    if start_index != -1:
        return row['Content'][start_index:]
    else:
      pass

In [ ]:
df['Extracted_Content'] = df.apply(extract_substring, axis=1)
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Extracted_Content
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,None
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
2,Plan Risk Management: Inputs,402,11.1.1,404,• Project charter11.1 Plan RiskManagementEn...,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...
3,Plan Risk Management: Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNI...,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...
4,Plan Risk Management: Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


In [ ]:
df['Extracted_Content'].loc[23] = '11.6.2.1 EXPERT JUDGMENT Described in Section 4.1.2.1. Expertise should be considered from individuals or groups with specialized knowledge to validate or modify risk responses if necessary, and decide how to implement them in the most efficient and effective manner. 11.6.2.2 INTERPERSONAL AND TEAM SKILLS Interpersonal and team skills that can be used for this process include but are not limited to influencing. Some risk response actions may be owned by people outside the immediate project team or who have other competing demands. The project manager or person responsible for facilitating the risk process may need to exercise influencing (see Section 9.5.2.1) to encourage nominated risk owners to take necessary action where required. 11.6.2.3 PROJECT MANAGEMENT INFORMATION SYSTEM (PMIS) Described in Section 4.3.2.2. Project management information systems can include schedule, resource, and cost software to ensure that agreed-upon risk response plans and their associated activities are integrated into the project alongside other project activities. 11.6.3 IMPLEMENT RISK RESPONSES: OUTPUTS 11.6.3.1 CHANGE REQUESTS Described in Section 4.3.3.4. Implementation of risk responses may result in a change request to the cost and schedule baselines or other components of the project management plan. Change requests are processed for review and disposition through the Perform Integrated Change Control process (Section 4.6).'
df['Extracted_Content'].loc[0] = df['Content'].loc[0]

In [ ]:
df['Extracted_Content'].isna().sum()

0

In [ ]:
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Extracted_Content
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11PROJECT RISK MANAGEMENTProject Risk Manageme...
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
2,Plan Risk Management: Inputs,402,11.1.1,404,• Project charter11.1 Plan RiskManagementEn...,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...
3,Plan Risk Management: Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNI...,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...
4,Plan Risk Management: Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


In [ ]:
def verifying_with_all_contents(df):
    for i, row in df.iterrows():
        print(f'For the {i}-th Content')
        title = row['Section'] + ' ' + row['Title'].upper()
        start = row['Starting_Page_Number']
        end = row['Ending_Page_Number']
        print(f'- With the Title = {title}')
        print(f'Starting in the Page : {start} Ending in the Page : {end}')
        print('- Content : ')
        print(row['Extracted_Content'])
        print('----------------')

verifying_with_all_contents(df)

For the 0-th Content
- With the Title = 11 PROJECT RISK MANAGEMENT
Starting in the Page : 395 Ending in the Page : 401
- Content : 
11PROJECT RISK MANAGEMENTProject Risk Management includes the processes of conducting risk management planning identification analysis response planning response implementation and monitoring risk on a project. The objectives of project risk management are to increase the probability and/or impact of positive risks and to decrease the probability and/or impact of negative risks in order to optimize the chances of project success.The Project Risk Management processes are:11.1 Plan Risk Management—The process of defining how to conduct risk management activities for a project.11.2 Identify Risks—The process of identifying individual project risks as well as sources of overall project risk and documenting their characteristics.11.3 Perform Qualitative Risk Analysis—The process of prioritizing individual project risks for further analysis or action by assessin

In [ ]:
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Extracted_Content
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11PROJECT RISK MANAGEMENTProject Risk Manageme...
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
2,Plan Risk Management: Inputs,402,11.1.1,404,• Project charter11.1 Plan RiskManagementEn...,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...
3,Plan Risk Management: Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNI...,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...
4,Plan Risk Management: Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


### Removing Figures

In [ ]:
# @title Saving Contents in Text files to remove Figures Manually
output_directory = '/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_CONTENTS1'
os.makedirs(output_directory, exist_ok=True)
def saving_each_extracted_content_in_a_text_file(df):
  for index, row in df.iterrows():
    file_name = f"{row['Section']}_{row['Title'].upper()}_{row['Starting_Page_Number']}_{row['Ending_Page_Number']}.txt"
    file_path = os.path.join(output_directory,file_name)
    with open(file_path, 'w') as file:
      file.write(row['Extracted_Content'])
saving_each_extracted_content_in_a_text_file(df)
df.drop('Content', axis=1, inplace = True)
df.drop('Extracted_Content', axis=1, inplace=True)

In [ ]:
# @title cleaning file names to get the content easily
def remove_digits_from_filenames(directory_path):
    if not os.path.exists(directory_path):
        print(f"Directory not found: {directory_path}")
        return
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            clean_filename = re.sub(r'\d+\.\d+', '', filename)
            clean_filename = re.sub(r'\.\d+', '', filename)
            clean_filename = re.sub(r'_', '', filename)
            clean_filename = re.sub(r'\d', '', filename)
            new_file_path = os.path.join(directory_path, clean_filename)
            os.rename(file_path, new_file_path)
remove_digits_from_filenames(r'/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_CONTENTS1')

In [ ]:
# @title getting Contents without figures from text files
df['Title'] = df['Title'].str.replace(':','')
def add_clean_content_to_dataframe(df, directory_path):
    if not os.path.exists(directory_path):
        print(f"Directory not found: {directory_path}")
        return
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            title = os.path.splitext(filename)[0]
            matching_row = df[df['Title'].str.lower()  == title.lower()]
            if not matching_row.empty:
                with open(file_path, 'r') as file:
                    content = file.read()
                df.loc[matching_row.index[0], 'Content'] = content
add_clean_content_to_dataframe(df, r'/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_CONTENTS')

### PMBOK CLEAN DATAFRAME

In [ ]:
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENT Plan Risk Management...
2,Plan Risk Management Inputs,402,11.1.1,404,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...
3,Plan Risk Management Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...
4,Plan Risk Management Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...


### NLP (Natural Language Processing)

#### Lower Case And Removing Stop Words

In NLP, we remove stop words and lower case the text for two primary reasons:

**1- Stop Words Removal:** Stop words are common words like "and," "the," "in," etc., that don't carry significant meaning for understanding the context. Removing them reduces the dimensionality of the text data and can improve processing efficiency. It also focuses the analysis on the more meaningful words.

**2- Lowercasing:** Lowercasing all words ensures uniformity and consistency. It prevents the model from treating words in different cases (e.g., "Risk" and "risk") as distinct tokens, which is important for accurate text analysis.

In [ ]:
def clean_text(text):
    # Apply the provided cleaning code
    text = re.sub(r'\be\.g\b', 'for example', text)
    text = re.sub(r',', '', text)
    text = ' '.join([word.lower() for word in text.split() if word.lower() not in stop]) # Lowercase + removing stop words
    return text

In [ ]:
df['Clean_Content[no_stop_words]'] = df['Content'].apply(clean_text)

In [ ]:
df.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/clean_contents1.csv', index=False) # Saving dataframe incase we need it later

#### Sentence Tokenization and Removing Redanduncy
* We used sentence tokenization to split text into sentences because it helps in analyzing and processing text at a finer level, making it easier to extract meaning from individual statements.
* Lemmatization is used to reduce words to their base or dictionary form (lemma), which standardizes variations of a word and ensures that different forms of a word are treated as the same token. This is crucial for improving accuracy in text analysis and information retrieval. (this helped to remove redundancy because we have encountered issues like => "risks" != "risk")

In [5]:
df_pmbok = pd.read_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/clean_contents1.csv')

In [6]:
def remove_repeated_and_plural_words(list_of_sentences):
    unique_sentences = []
    lemmatizer = WordNetLemmatizer()
    for sentence in list_of_sentences:
        words = sentence.split()
        unique_words = []
        for word in words:
            lemma_word = lemmatizer.lemmatize(word)
            if lemma_word not in unique_words:
                unique_words.append(lemma_word)
        unique_sentence = ' '.join(unique_words)
        unique_sentences.append(unique_sentence)
    return unique_sentences

In [7]:
#Sentence Tokenization using Regex because we have sections that are numbers concatenated by dots e.g: 11.2.3
df_pmbok['Sentences'] = df_pmbok['Clean_Content[no_stop_words]'].apply(lambda x: re.split(r'(?<!\d)\.(?!\d)', x))
df_pmbok['Sentences'] = df_pmbok['Sentences'].apply(remove_repeated_and_plural_words)
df_pmbok.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Clean_Content[no_stop_words],Sentences
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,[11project risk managementproject management i...
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,[11.1 plan risk management process defining co...
2,Plan Risk Management Inputs,402,11.1.1,404,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,[11.1.1 plan risk management: inputs11.1.1.1 p...
3,Plan Risk Management Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques ...,[11.1.2 plan risk management: tool technique 1...
4,Plan Risk Management Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...,[11.1.3 plan risk management: outputs11.1.3.1 ...


#### POS TAGGING
POS (Part-of-Speech) tagging is used to label words in a text with their grammatical category (e.g., noun, verb, adjective) to understand the syntactic structure and relationships within sentences.
In our case we need the NOUNs to use as Concepts and VERBS as relationships (so we can create graphs)



In [8]:
def pos_tagging(list_of_sentences):
    pos_tagged_sentences = []
    for sentence in list_of_sentences:
        try:
            words = nltk.word_tokenize(sentence)
            pos_tags = nltk.pos_tag(words)
            pos_tagged_sentences.append(pos_tags)
        except Exception as e:
            print(f"Error processing sentence: {sentence}")
            continue  # Skip the sentence with errors
    return pos_tagged_sentences

In [9]:
df_pmbok['POS_Tags'] = df_pmbok['Sentences'].apply(pos_tagging)

In [ ]:
df_pmbok.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Clean_Content[no_stop_words],Sentences,POS_Tags
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,[11project risk managementproject management i...,"[[(11project, CD), (risk, NN), (managementproj..."
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,[11.1 plan risk management process defining co...,"[[(11.1, CD), (plan, NN), (risk, NN), (managem..."
2,Plan Risk Management Inputs,402,11.1.1,404,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,[11.1.1 plan risk management: inputs11.1.1.1 p...,"[[(11.1.1, CD), (plan, NN), (risk, NN), (manag..."
3,Plan Risk Management Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques ...,[11.1.2 plan risk management: tool technique 1...,"[[(11.1.2, CD), (plan, NN), (risk, NN), (manag..."
4,Plan Risk Management Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...,[11.1.3 plan risk management: outputs11.1.3.1 ...,"[[(11.1.3, CD), (plan, NN), (risk, NN), (manag..."


#### Extracting KeyWords Using KeyBert
Extracting keywords using KeyBert helps identify important concepts and themes in text, which is essential for creating a meaningful conceptual graph. KeyBert leverages contextual embeddings to find contextually relevant keywords, ensuring that the graph is based on the most significant terms. This process enhances the quality of the recommendation system, as it relies on a well-structured and semantically rich graph to make informed suggestions.

In [10]:
kw_model = KeyBERT(model='all-mpnet-base-v2')
def extract_concepts_keybert(text):
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1,3),
        stop_words='english',
        highlight=False,
        top_n=10
    )
    return [keyword[0] for keyword in keywords]

In [11]:
df_pmbok['KeyWords_KeyBert'] = df_pmbok['Clean_Content[no_stop_words]'].apply(extract_concepts_keybert)
df_pmbok[['Title', 'KeyWords_KeyBert']]

,Title,KeyWords_KeyBert
0,PROJECT RISK MANAGEMENT,"[risk managementall projects, project risk man..."
1,Plan Risk Management,"[project risk management, risk management proc..."
2,Plan Risk Management Inputs,"[project risk management, managing risk projec..."
3,Plan Risk Management Tools and Techniques,"[plan risk management, conducting risk managem..."
4,Plan Risk Management Outputs,"[project risk management, managing risk projec..."
5,Identify Risks,"[project risk associated, project risks level,..."
6,Identify Risks Inputs,"[project risk introduce, risks project details..."
7,Identify Risks Tools and Techniques,"[risk project threats, risks identified brains..."
8,Identify Risks Outputs,"[project risk register, project risk summary, ..."
9,Perform Qualitative Risk Analysis,"[project risks analysis, project risks helps, ..."


#### Extracting KeyWords Using TFIDF
Extracting keywords using TF-IDF (Term Frequency-Inverse Document Frequency) involves ranking words based on their importance in a document corpus.

In [ ]:
def extract_top_keywords(sentences_list):
    text = ' '.join(sentences_list)
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])
    feature_names = tfidf_vectorizer.get_feature_names_out()
    top_keywords = [feature_names[i] for i in tfidf_matrix[0].indices]
    return top_keywords
df_pmbok['Keyword_TFIDF'] = df_pmbok['Sentences'].apply(extract_top_keywords)

In [20]:
df_pmbok.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK.csv') # Saving dataset incase we need it later

In [ ]:
df_pmbok.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Clean_Content[no_stop_words],Sentences,POS_Tags,KeyWords_KeyBert,Keyword_TFIDF
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,[11project risk managementproject management i...,"[[(11project, CD), (risk, NN), (managementproj...","[risk managementall projects, project risk man...","[current, reprioritized, regularly, updated, d..."
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,[11.1 plan risk management process defining co...,"[[(11.1, CD), (plan, NN), (risk, NN), (managem...","[project risk management, risk management proc...","[modification, requires, determines, effective..."
2,Plan Risk Management Inputs,402,11.1.1,404,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,[11.1.1 plan risk management: inputs11.1.1.1 p...,"[[(11.1.1, CD), (plan, NN), (risk, NN), (manag...","[project risk management, managing risk projec...","[similar, previous, repository, learned, andle..."
3,Plan Risk Management Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques ...,[11.1.2 plan risk management: tool technique 1...,"[[(11.1.2, CD), (plan, NN), (risk, NN), (manag...","[plan risk management, conducting risk managem...","[see, documented, defined, activity, conductin..."
4,Plan Risk Management Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...,[11.1.3 plan risk management: outputs11.1.3.1 ...,"[[(11.1.3, CD), (plan, NN), (risk, NN), (manag...","[project risk management, managing risk projec...","[audited, document, tracking, output, well, re..."


#### Extracting Synonyms using Spacy
We extract synonyms to enhance information retrieval and expand the search scope. By comparing similarity between titles and keywords using spaCy, if the similarity is above 0.7, we save the term as a synonym. This helps broaden the query results, improving the chances of finding relevant information. In contrast, if the similarity is below 0.7, we retain it as a keyword to maintain specificity in search queries. This approach strikes a balance between precision and recall in information retrieval.

In [ ]:
synonym_threshold = 0.7
def classify_concepts(concept, title_doc):
    synonyms = []
    keywords = []
    if isinstance(concept, str):
        concept_doc = nlp(concept) # treat the key word with spacy
        similarity_score = title_doc.similarity(concept_doc)
        if similarity_score >= synonym_threshold:
            synonyms.append(concept)
        else:
            keywords.append(concept)
    elif isinstance(concept, list):
        for item in concept:
            s, k = classify_concepts(item, title_doc)
            synonyms.extend(s)
            keywords.extend(k)
    return synonyms, keywords

synonyms_list = []
keywords_list = []

for index, row in df_pmbok.iterrows():
    title = row['Title']
    concepts = row['KeyWords_KeyBert']
    if title and concepts:
        title_doc = nlp(title)
        synonyms, keywords = classify_concepts(concepts, title_doc)
        synonyms = list(set(synonyms))
        keywords = list(set(keywords))
        if not synonyms:
            synonyms = ['None']
        if not keywords:
            keywords = ['None']
        synonyms_list.append(", ".join(synonyms))
        keywords_list.append(", ".join(keywords))
    else:
        synonyms_list.append('None')
        keywords_list.append('None')
df_pmbok['Synonyms_Spacy'] = synonyms_list
df_pmbok.head(6)

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Clean_Content[no_stop_words],Sentences,POS_Tags,KeyWords_KeyBert,Keyword_TFIDF,Synonyms,Keywords,Concepts,Synonyms_Spacy
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,[11project risk managementproject management i...,"[[(11project, CD), (risk, NN), (managementproj...","[risk managementall projects, project risk man...","[current, reprioritized, regularly, updated, d...","project risk managementall, project management...","project risks analysis, risk managementall pro...","[[risk, managementproject, management, process...","project risk managementall, project management..."
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,[11.1 plan risk management process defining co...,"[[(11.1, CD), (plan, NN), (risk, NN), (managem...","[project risk management, risk management proc...","[modification, requires, determines, effective...","determines project risk, management plan risk,...",risks importance project,"[[plan, risk, management, process, conduct, ac...","determines project risk, management plan risk,..."
2,Plan Risk Management Inputs,402,11.1.1,404,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,[11.1.1 plan risk management: inputs11.1.1.1 p...,"[[(11.1.1, CD), (plan, NN), (risk, NN), (manag...","[project risk management, managing risk projec...","[similar, previous, repository, learned, andle...","risk management process, plan risk management,...","managing risk project, project risk, risk proj...","[[plan, risk, management, inputs11.1.1.1, proj...","risk management process, plan risk management,..."
3,Plan Risk Management Tools and Techniques,404,11.1.2,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques ...,[11.1.2 plan risk management: tool technique 1...,"[[(11.1.2, CD), (plan, NN), (risk, NN), (manag...","[plan risk management, conducting risk managem...","[see, documented, defined, activity, conductin...",None,"manage risk management, plans conducting risk,...","[[plan, risk, management, tool, technique, exp...",None
4,Plan Risk Management Outputs,405,11.1.3,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...,[11.1.3 plan risk management: outputs11.1.3.1 ...,"[[(11.1.3, CD), (plan, NN), (risk, NN), (manag...","[project risk management, managing risk projec...","[audited, document, tracking, output, well, re...","risk project methodology, risk management plan...","managing risk project, project risks definitio...","[[plan, risk, management, management, planthe,...","risk project methodology, risk management plan..."
5,Identify Risks,409,11.2,411,11.2 IDENTIFY RISKSIdentify Risks is the proce...,11.2 identify risksidentify risks process iden...,[11.2 identify risksidentify risk process iden...,"[[(11.2, CD), (identify, NN), (risksidentify, ...","[project risk associated, project risks level,...","[defined, situation, vary, participation, iter...",identify risks process,"project risks, project risk participants, proj...","[[identify, risksidentify, risk, process, proj...",identify risks process


#### Extracting KeyWords Using LDA TFIDF
 LDA TF-IDF combines TF-IDF with topic modeling (Latent Dirichlet Allocation) to find keywords with both term frequency and topic relevance.

In [ ]:
def extract_ngrams(tokens):
    ngram_list = []
    for sentence_tokens in tokens:
        for n in range(1, 4):  # Extract n-grams from 1 to 3 words
            ngram_list.extend(list(ngrams(sentence_tokens, n)))
    return ngram_list

vectorizer = TfidfVectorizer(max_df=0.85, max_features=1000, stop_words='english', ngram_range=(1, 3))
X = vectorizer.fit_transform(df_pmbok['Content'])
num_topics = 5

concepts_list = []

for index, row in df_pmbok.iterrows():
    section_text = ' '.join(row['Sentences'])
    X_section = vectorizer.transform([section_text])

    lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda.fit(X_section)

    feature_names = vectorizer.get_feature_names_out()
    top_keywords_idx = lda.components_[0].argsort()[:-10 - 1:-1]
    top_keywords = [feature_names[i] for i in top_keywords_idx]

    concepts_list.append(top_keywords)
df_pmbok['KeyWords_LDA_TFIDF'] = concepts_list
df_pmbok.head(3)

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Clean_Content[no_stop_words],Sentences,POS_Tags,KeyWords_KeyBert,Keyword_TFIDF,Concepts,Synonyms_Spacy,KeyWords_LDA_TFIDF
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,[11project risk managementproject management i...,"[[(11project, CD), (risk, NN), (managementproj...","[risk managementall projects, project risk man...","[current, reprioritized, regularly, updated, d...","[[risk, managementproject, management, process...","project risk managementall, project management...","[objective, event, managed, addressed, positiv..."
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,[11.1 plan risk management process defining co...,"[[(11.1, CD), (plan, NN), (risk, NN), (managem...","[project risk management, risk management proc...","[modification, requires, determines, effective...","[[plan, risk, management, process, conduct, ac...","determines project risk, management plan risk,...","[risk management, plan risk management, 11 pla..."
2,Plan Risk Management Inputs,402,11.1.1,404,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,[11.1.1 plan risk management: inputs11.1.1.1 p...,"[[(11.1.1, CD), (plan, NN), (risk, NN), (manag...","[project risk management, managing risk projec...","[similar, previous, repository, learned, andle...","[[plan, risk, management, inputs11.1.1.1, proj...","risk management process, plan risk management,...","[project management, considered, stakeholders,..."


In [ ]:
df_pmbok.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK.csv', index=False) # Saving dataset incase we need it later

#### Cleaning Dataframe
we noticed that KeyBert outperformed TFIDF and LDA TFIDF methods as it leverages contextual embeddings to identify contextually relevant and semantically rich keywords, making it more effective for capturing the essence of the text and generating the most meaningful keywords.

In [ ]:
# Cleaning Dataframe
pmbok = pd.read_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK.csv')
pmbok.head(3)

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content,Clean_Content[no_stop_words],Sentences,POS_Tags,KeyWords_KeyBert,Keyword_TFIDF,Synonyms_Spacy,KeyWords_LDA_TFIDF,Concepts[NN]
0,PROJECT RISK MANAGEMENT,395,11,401,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,['11project risk managementproject management ...,"[[('11project', 'CD'), ('risk', 'NN'), ('manag...","['risk managementall projects', 'project risk ...","['current', 'reprioritized', 'regularly', 'upd...","project risk managementall, project management...","['objective', 'event', 'managed', 'addressed',...","[['risk', 'managementproject', 'management', '..."
1,Plan Risk Management,401,11.1,402,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,['11.1 plan risk management process defining c...,"[[('11.1', 'CD'), ('plan', 'NN'), ('risk', 'NN...","['project risk management', 'risk management p...","['modification', 'requires', 'determines', 'ef...","determines project risk, management plan risk,...","['risk management', 'plan risk management', '1...","[['plan', 'risk', 'management', 'process', 'co..."
2,Plan Risk Management Inputs,402,11.1.1,404,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,['11.1.1 plan risk management: inputs11.1.1.1 ...,"[[('11.1.1', 'CD'), ('plan', 'NN'), ('risk', '...","['project risk management', 'managing risk pro...","['similar', 'previous', 'repository', 'learned...","risk management process, plan risk management,...","['project management', 'considered', 'stakehol...","[['plan', 'risk', 'management', 'inputs11.1.1...."


In [ ]:
pmbok['Title'] = pmbok['Section'] + ' ' + pmbok['Title']
#we dont need starting page and ending page numbers anymore and section can be included in the title
pmbok.drop(['Keyword_TFIDF','KeyWords_LDA_TFIDF','Starting_Page_Number', 'Ending_Page_Number','Section'],axis = 1 , inplace = True)
pmbok.rename(columns={'Concepts[NN]' : 'NOUNS[from_POS_TAGS]'}, inplace=True)
pmbok.head()

,Title,Content,Clean_Content[no_stop_words],Sentences,POS_Tags,KeyWords_KeyBert,Synonyms_Spacy,NOUNS[from_POS_TAGS]
0,11 PROJECT RISK MANAGEMENT,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,['11project risk managementproject management ...,"[[('11project', 'CD'), ('risk', 'NN'), ('manag...","['risk managementall projects', 'project risk ...","project risk managementall, project management...","[['risk', 'managementproject', 'management', '..."
1,11.1 Plan Risk Management,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,['11.1 plan risk management process defining c...,"[[('11.1', 'CD'), ('plan', 'NN'), ('risk', 'NN...","['project risk management', 'risk management p...","determines project risk, management plan risk,...","[['plan', 'risk', 'management', 'process', 'co..."
2,11.1.1 Plan Risk Management Inputs,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,['11.1.1 plan risk management: inputs11.1.1.1 ...,"[[('11.1.1', 'CD'), ('plan', 'NN'), ('risk', '...","['project risk management', 'managing risk pro...","risk management process, plan risk management,...","[['plan', 'risk', 'management', 'inputs11.1.1...."
3,11.1.2 Plan Risk Management Tools and Techniques,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques ...,['11.1.2 plan risk management: tool technique ...,"[[('11.1.2', 'CD'), ('plan', 'NN'), ('risk', '...","['plan risk management', 'conducting risk mana...",None,"[['plan', 'risk', 'management', 'tool', 'techn..."
4,11.1.3 Plan Risk Management Outputs,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...,['11.1.3 plan risk management: outputs11.1.3.1...,"[[('11.1.3', 'CD'), ('plan', 'NN'), ('risk', '...","['project risk management', 'managing risk pro...","risk project methodology, risk management plan...","[['plan', 'risk', 'management', 'management', ..."


#### Extracting Verbs Only

In [ ]:
l = []
for i in pmbok['POS_Tags']:
  l.append(ast.literal_eval(i))
print(l)

[[[('11project', 'CD'), ('risk', 'NN'), ('managementproject', 'NN'), ('management', 'NN'), ('includes', 'VBZ'), ('process', 'NN'), ('conducting', 'VBG'), ('planning', 'VBG'), ('identification', 'NN'), ('analysis', 'NN'), ('response', 'NN'), ('implementation', 'NN'), ('monitoring', 'NN'), ('project', 'NN')], [('objective', 'JJ'), ('project', 'NN'), ('risk', 'NN'), ('management', 'NN'), ('increase', 'NN'), ('probability', 'NN'), ('and/or', 'JJ'), ('impact', 'NN'), ('positive', 'JJ'), ('decrease', 'NN'), ('negative', 'JJ'), ('order', 'NN'), ('optimize', 'NNS'), ('chance', 'NN'), ('success', 'NN')], [('the', 'DT'), ('project', 'NN'), ('risk', 'NN'), ('management', 'NN'), ('process', 'NN'), ('are:11.1', 'NN'), ('plan', 'NN'), ('management—the', 'VBP'), ('defining', 'VBG'), ('conduct', 'NN'), ('activity', 'NN'), ('project.11.2', 'VBP'), ('identify', 'VB'), ('risks—the', 'NN'), ('identifying', 'VBG'), ('individual', 'JJ'), ('well', 'RB'), ('source', 'NN'), ('overall', 'JJ'), ('documenting', '

In [ ]:
verb_lists = []
vbs = ['VBG', 'VBN', 'VBZ']
for i in l:
    each_sentence_verbs = []
    for j in i:
        for k, z in j:
            if z in vbs:
                each_sentence_verbs.append(k)
    verb_lists.append(each_sentence_verbs)
for i in verb_lists:
    print(i)


['includes', 'conducting', 'planning', 'defining', 'identifying', 'prioritizing', 'assessing', 'analyzing', 'combined', 'identified', 'developing', 'selecting', 'agreeing', 'implementing', 'monitoring', 'tracking', 'evaluating', 'provides', 'undertaking', 'varying', 'responding', 'changing', 'balancing', 'defined', 'related', 'exists', 'contains', 'arises', 'follows', 'occurs', 'arising', 'including', 'representing', 'avoiding', 'mitigating', 'captured', 'reducing', 'promoting', 'maximizing', 'achieving', 'planning', 'shaping', 'defined', 'stated', 'communicated', 'emerging', 'considered', 'emerging', 'limited', 'increasing', 'identified', 'exists', 'planned', 'testing', 'expected', 's', 'using', 'defining', 'understanding', 'filling', 'obtaining', 'benchmarking', 'prototyping', 'becoming', 'growing', 'developing', 'requires', 'maintaining', 'including', 'empowered', 'done', 'adjusted', 'integrated', 'owned', 'managed', 'coordinated', 'ensures', 'providing', 'given', 'tailoring', 'tail

In [ ]:
pmbok['VERBS[from_POS_TAGS]'] = verb_lists
pmbok.head()

,Title,Content,Clean_Content[no_stop_words],Sentences,POS_Tags,KeyWords_KeyBert,Synonyms_Spacy,VERBS[from_POS_TAGS]
0,11 PROJECT RISK MANAGEMENT,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,['11project risk managementproject management ...,"[[('11project', 'CD'), ('risk', 'NN'), ('manag...","['risk managementall projects', 'project risk ...","project risk managementall, project management...","[includes, conducting, planning, defining, ide..."
1,11.1 Plan Risk Management,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,['11.1 plan risk management process defining c...,"[[('11.1', 'CD'), ('plan', 'NN'), ('risk', 'NN...","['project risk management', 'risk management p...","determines project risk, management plan risk,...","[defining, ensures, depicted, completed, deter..."
2,11.1.1 Plan Risk Management Inputs,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,['11.1.1 plan risk management: inputs11.1.1.1 ...,"[[('11.1.1', 'CD'), ('plan', 'NN'), ('risk', '...","['project risk management', 'managing risk pro...","risk management process, plan risk management,...","[taken, considered, limited, s, provides, dete..."
3,11.1.2 Plan Risk Management Tools and Techniques,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques ...,['11.1.2 plan risk management: tool technique ...,"[[('11.1.2', 'CD'), ('plan', 'NN'), ('risk', '...","['plan risk management', 'conducting risk mana...",None,"[considered, following, s, managing, including..."
4,11.1.3 Plan Risk Management Outputs,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...,['11.1.3 plan risk management: outputs11.1.3.1...,"[[('11.1.3', 'CD'), ('plan', 'NN'), ('risk', '...","['project risk management', 'managing risk pro...","risk project methodology, risk management plan...","[describes, following, managing, used, related..."


#### Extracting Nouns Only

In [ ]:
pmbok.drop('NOUNS[from_POS_TAGS]', axis=1, inplace=True)

In [ ]:
nn_lists = []
nns = ['NN','NNS']
for i in l:
    each_sentence_nn = []
    for j in i:
        for k, z in j:
            if z in nns:
                each_sentence_nn.append(k)
    nn_lists.append(each_sentence_nn)
for i in nn_lists:
    print(i)

['risk', 'managementproject', 'management', 'process', 'identification', 'analysis', 'response', 'implementation', 'monitoring', 'project', 'project', 'risk', 'management', 'increase', 'probability', 'impact', 'decrease', 'order', 'optimize', 'chance', 'success', 'project', 'risk', 'management', 'process', 'are:11.1', 'plan', 'conduct', 'activity', 'risks—the', 'source', 'documenting', 'characteristics.11.3', 'perform', 'analysis—the', 'analysis', 'action', 'probability', 'occurrence', 'impact', 'effect', 'uncertainty', 'option', 'strategy', 'address', 'exposure', 'treat', 'response', 'plans.11.7', 'monitor', 'implementation', 'effectiveness', 'figure', 'overview', 'project', 'risk', 'management', 'process', 'project', 'management', 'risk', 'process', 'interface', 'practice', 'way', 'pmbok®', 'concept', 'project', 'risk', 'managementall', 'complexity', 'aim', 'deliver', 'benefit', 'constraint', 'assumption', 'expectation', 'organization', 'choose', 'project', 'risk', 'manner', 'order',

In [ ]:
pmbok['NOUNS[from_POS_TAGS]'] = nn_lists
pmbok.head()

,Title,Content,Clean_Content[no_stop_words],Sentences,POS_Tags,KeyWords_KeyBert,Synonyms_Spacy,VERBS[from_POS_TAGS],NOUNS[from_POS_TAGS]
0,11 PROJECT RISK MANAGEMENT,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,['11project risk managementproject management ...,"[[('11project', 'CD'), ('risk', 'NN'), ('manag...","['risk managementall projects', 'project risk ...","project risk managementall, project management...","[includes, conducting, planning, defining, ide...","[risk, managementproject, management, process,..."
1,11.1 Plan Risk Management,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,['11.1 plan risk management process defining c...,"[[('11.1', 'CD'), ('plan', 'NN'), ('risk', 'NN...","['project risk management', 'risk management p...","determines project risk, management plan risk,...","[defining, ensures, depicted, completed, deter...","[plan, risk, management, process, conduct, act..."
2,11.1.1 Plan Risk Management Inputs,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,['11.1.1 plan risk management: inputs11.1.1.1 ...,"[[('11.1.1', 'CD'), ('plan', 'NN'), ('risk', '...","['project risk management', 'managing risk pro...","risk management process, plan risk management,...","[taken, considered, limited, s, provides, dete...","[plan, risk, management, inputs11.1.1.1, proje..."
3,11.1.2 Plan Risk Management Tools and Techniques,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques ...,['11.1.2 plan risk management: tool technique ...,"[[('11.1.2', 'CD'), ('plan', 'NN'), ('risk', '...","['plan risk management', 'conducting risk mana...",None,"[considered, following, s, managing, including...","[plan, risk, management, tool, technique, expe..."
4,11.1.3 Plan Risk Management Outputs,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...,['11.1.3 plan risk management: outputs11.1.3.1...,"[[('11.1.3', 'CD'), ('plan', 'NN'), ('risk', '...","['project risk management', 'managing risk pro...","risk project methodology, risk management plan...","[describes, following, managing, used, related...","[plan, risk, management, management, planthe, ..."


In [ ]:
pmbok.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_final_1.0.csv', index=False)

#### Extracting Synonyms Using SBert

In [76]:
pmbok = pd.read_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_final_1.0.csv')

In [77]:
def actual_lists_from_string_back_list(df,column):
    sent = []
    for i in range(len(df)):
        input_string = df[column][i].replace("'", '"')
        sent.append(eval(input_string))
    return sent
pmbok['Sentences'] = actual_lists_from_string_back_list(pmbok,'Sentences')
pmbok['KeyWords_KeyBert'] = actual_lists_from_string_back_list(pmbok,'KeyWords_KeyBert')
pmbok.head()

,Title,Content,Clean_Content[no_stop_words],Sentences,POS_Tags,KeyWords_KeyBert,Synonyms_Spacy,VERBS[from_POS_TAGS],NOUNS[from_POS_TAGS]
0,11 PROJECT RISK MANAGEMENT,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,[11project risk managementproject management i...,"[[('11project', 'CD'), ('risk', 'NN'), ('manag...","[risk managementall projects, project risk man...","project risk managementall, project management...","['includes', 'conducting', 'planning', 'defini...","['risk', 'managementproject', 'management', 'p..."
1,11.1 Plan Risk Management,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,[11.1 plan risk management process defining co...,"[[('11.1', 'CD'), ('plan', 'NN'), ('risk', 'NN...","[project risk management, risk management proc...","determines project risk, management plan risk,...","['defining', 'ensures', 'depicted', 'completed...","['plan', 'risk', 'management', 'process', 'con..."
2,11.1.1 Plan Risk Management Inputs,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,[11.1.1 plan risk management: inputs11.1.1.1 p...,"[[('11.1.1', 'CD'), ('plan', 'NN'), ('risk', '...","[project risk management, managing risk projec...","risk management process, plan risk management,...","['taken', 'considered', 'limited', 's', 'provi...","['plan', 'risk', 'management', 'inputs11.1.1.1..."
3,11.1.2 Plan Risk Management Tools and Techniques,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques ...,[11.1.2 plan risk management: tool technique 1...,"[[('11.1.2', 'CD'), ('plan', 'NN'), ('risk', '...","[plan risk management, conducting risk managem...",None,"['considered', 'following', 's', 'managing', '...","['plan', 'risk', 'management', 'tool', 'techni..."
4,11.1.3 Plan Risk Management Outputs,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...,[11.1.3 plan risk management: outputs11.1.3.1 ...,"[[('11.1.3', 'CD'), ('plan', 'NN'), ('risk', '...","[project risk management, managing risk projec...","risk project methodology, risk management plan...","['describes', 'following', 'managing', 'used',...","['plan', 'risk', 'management', 'management', '..."


In [95]:
def Synonym_extraction_SBert(df,title_column,keywords_column):
  All_Synonyms = []
  All_KeyWords = []
  for index, row in df.iterrows():
      sentences1 = [row[title_column]]
      sentences2 = row[keywords_column]
      embeddings1 = Sbert.encode(sentences1, convert_to_tensor=True)
      test = []
      synonym = []
      keywords = []
      for i in sentences2:
          embeddings2 = Sbert.encode(i, convert_to_tensor=True)
          cosine_scores = util.cos_sim(embeddings1, embeddings2)
          score = cosine_scores[0][0].item()
          if score >= 0.8:
              synonym.append(i)
          else:
              keywords.append(i)
          test.append(f'{sentences1[0]} <=> {i} Has the score: {score}')
      All_Synonyms.append(synonym)
      All_KeyWords.append(keywords)
  return All_Synonyms, All_KeyWords

In [97]:
pmbok['Synonyms_SBert'],pmbok['KeyWords_SBert'] = Synonym_extraction_SBert(pmbok,'Title','KeyWords_KeyBert')

In [90]:
pmbok.head()

,Title,Content,Clean_Content[no_stop_words],Sentences,POS_Tags,KeyWords_KeyBert,Synonyms_Spacy,VERBS[from_POS_TAGS],NOUNS[from_POS_TAGS],Synonyms_SBert,KeyWords_SBert
0,11 PROJECT RISK MANAGEMENT,11PROJECT RISK MANAGEMENTProject Risk Manageme...,11project risk managementproject risk manageme...,[11project risk managementproject management i...,"[[('11project', 'CD'), ('risk', 'NN'), ('manag...","[risk managementall projects, project risk man...","project risk managementall, project management...","['includes', 'conducting', 'planning', 'defini...","['risk', 'managementproject', 'management', 'p...","[risk managementall projects, project risk man...",[risk managementproject risk]
1,11.1 Plan Risk Management,11.1 PLAN RISK MANAGEMENT Plan Risk Management...,11.1 plan risk management plan risk management...,[11.1 plan risk management process defining co...,"[[('11.1', 'CD'), ('plan', 'NN'), ('risk', 'NN...","[project risk management, risk management proc...","determines project risk, management plan risk,...","['defining', 'ensures', 'depicted', 'completed...","['plan', 'risk', 'management', 'process', 'con...",[plan risk management],"[project risk management, risk management proc..."
2,11.1.1 Plan Risk Management Inputs,11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PR...,11.1.1 plan risk management: inputs11.1.1.1 pr...,[11.1.1 plan risk management: inputs11.1.1.1 p...,"[[('11.1.1', 'CD'), ('plan', 'NN'), ('risk', '...","[project risk management, managing risk projec...","risk management process, plan risk management,...","['taken', 'considered', 'limited', 's', 'provi...","['plan', 'risk', 'management', 'inputs11.1.1.1...",[],"[project risk management, managing risk projec..."
3,11.1.2 Plan Risk Management Tools and Techniques,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNIQ...,11.1.2 plan risk management: tools techniques ...,[11.1.2 plan risk management: tool technique 1...,"[[('11.1.2', 'CD'), ('plan', 'NN'), ('risk', '...","[plan risk management, conducting risk managem...",None,"['considered', 'following', 's', 'managing', '...","['plan', 'risk', 'management', 'tool', 'techni...",[plan risk management],"[conducting risk management, risk management p..."
4,11.1.3 Plan Risk Management Outputs,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...,11.1.3 plan risk management: outputs11.1.3.1 r...,[11.1.3 plan risk management: outputs11.1.3.1 ...,"[[('11.1.3', 'CD'), ('plan', 'NN'), ('risk', '...","[project risk management, managing risk projec...","risk project methodology, risk management plan...","['describes', 'following', 'managing', 'used',...","['plan', 'risk', 'management', 'management', '...",[],"[project risk management, managing risk projec..."


In [91]:
pmbok.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_final_2.0.csv', index=False)